# Scatter plot map for Walcott 
- will use Rathbun scatter plot for reference 
- but will more detailed hovertext that includes specimen family name and year (but dates on file seem to be wrong)
- if there were fewer points or more time, could use mapbox
    - will us go.Scatter in plotly instead 
- using example from plotly 
    - link: https://plot.ly/pandas/scatter-plots-on-maps/

In [262]:
import pandas as pd 
import plotly.graph_objects as go
import numpy as np

### clean dataframe from original xlsx file
- need to create "sub" dataframe with only the columns I need for my scatter plot map 
- can use example from jones scatter plot map 

In [2]:
walcott_df = pd.read_csv('AWHI_Walcott_dataset.csv')
walcott_df.head()

,barcode,catalog_no,order,family,family_name,collector_name,year_collected,country,province_state,ezid
0,1160457,99194.0,Solanales,Hydrophyllaceae,Waterleaf family,C. D. Walcott,1900.0,United States,Colorado,http://n2t.net/ark:/65665/30a751787-3a18-4f16-...
1,3089925,80416.0,Apiales,Apiaceae,Umbellifers,C. D. Walcott,1900.0,United States,New Mexico,http://n2t.net/ark:/65665/3d8b58966-9a8b-4b3b-...
2,3088994,80403.0,Apiales,Apiaceae,Umbellifers,C. D. Walcott,1900.0,United States,New Mexico,http://n2t.net/ark:/65665/3c0f352f5-2101-4057-...
3,1805301,27505.0,Asterales,Asteraceae,Daisy family,C. D. Walcott,1900.0,United States,New Mexico,http://n2t.net/ark:/65665/3db47f2f9-1144-47d3-...
4,2180001,NaN,Asterales,Asteraceae,Daisy family,C. D. Walcott,1900.0,United States,New Mexico,http://n2t.net/ark:/65665/3cd015798-9922-4328-...


### Pulling information from the imported dataframe 
- this information will be useful for determining how to restructure the dataframe,<br>recreate sub dataframe, and figure out what type of plot and map will be most useful 
- instead of creating new csv files or eliminating information from the main csv file, it's better to make those changes using pandas 

In [3]:
walcott_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 10 columns):
barcode           219 non-null int64
catalog_no        209 non-null float64
order             219 non-null object
family            219 non-null object
family_name       219 non-null object
collector_name    219 non-null object
year_collected    193 non-null float64
country           219 non-null object
province_state    219 non-null object
ezid              219 non-null object
dtypes: float64(2), int64(1), object(7)
memory usage: 17.2+ KB


In [4]:
province_state = walcott_df.province_state.unique()
province_state

array(['Colorado', 'New Mexico', 'Alberta', 'Arizona', 'California',
       'Nevada', 'New Brunswick', 'Tennessee', 'Maryland',
       'District of Columbia', 'Oklahoma', 'Arkansas', 'Utah', 'Colima',
       'British Columbia', 'Ontario', 'Saskatchewan', 'Guánica',
       'South Carolina'], dtype=object)

In [5]:
#number of specimens collected by each collector
walcott_df['collector_name'].value_counts()

M. V. Walcott    172
C. D. Walcott     47
Name: collector_name, dtype: int64

In [6]:
# number of specimens collected each location
walcott_df['province_state'].value_counts()

British Columbia        94
Alberta                 50
California              15
New Mexico              13
Arizona                 11
Utah                     7
Tennessee                7
South Carolina           4
Colorado                 4
Maryland                 3
Nevada                   3
Oklahoma                 1
Arkansas                 1
Colima                   1
New Brunswick            1
District of Columbia     1
Saskatchewan             1
Ontario                  1
Guánica                  1
Name: province_state, dtype: int64

In [264]:
# I want to add a specimens list collected at each location for each collector then use this to add to the text option in the map
walcott_df.groupby(['province_state','collector_name','family_name']).size()

province_state    collector_name  family_name       
Alberta           C. D. Walcott   Cactus                 1
                  M. V. Walcott   Borages                3
                                  Cactus                 1
                                  Daisy family          11
                                  Gentians               6
                                  Heather family         4
                                  Honeysuckle family     1
                                  Madder family          1
                                  Orchids                2
                                  Pine family            2
                                  Primroses              2
                                  Sedges                 1
                                  Spleenworts            1
                                  Violet family          1
                                  Waterleaf family       3
                                  legumes               10
Ari

In [7]:
#number of specimens in each location for each collector
walcott_df.groupby(['province_state','collector_name']).size()

province_state        collector_name
Alberta               C. D. Walcott      1
                      M. V. Walcott     49
Arizona               C. D. Walcott      6
                      M. V. Walcott      5
Arkansas              C. D. Walcott      1
British Columbia      M. V. Walcott     94
California            C. D. Walcott      8
                      M. V. Walcott      7
Colima                M. V. Walcott      1
Colorado              C. D. Walcott      2
                      M. V. Walcott      2
District of Columbia  C. D. Walcott      1
Guánica               M. V. Walcott      1
Maryland              C. D. Walcott      3
Nevada                C. D. Walcott      3
New Brunswick         C. D. Walcott      1
New Mexico            C. D. Walcott     12
                      M. V. Walcott      1
Oklahoma              C. D. Walcott      1
Ontario               M. V. Walcott      1
Saskatchewan          M. V. Walcott      1
South Carolina        M. V. Walcott      4
Tennessee        

### sub dataframe location_walcott_df data use
- this dataframe will be used to create the trace for each collector to make their location
- each location will have a marker size relative to the ind_count number 

In [8]:
#creating dataframe for previous groupby function
location_walcott_df = walcott_df.groupby(['province_state','collector_name']).size().to_frame().reset_index()
#renaming last column in groupby function above 
location_walcott_df = location_walcott_df.rename(columns={0: 'ind_count'})
location_walcott_df 

,province_state,collector_name,ind_count
0,Alberta,C. D. Walcott,1
1,Alberta,M. V. Walcott,49
2,Arizona,C. D. Walcott,6
3,Arizona,M. V. Walcott,5
4,Arkansas,C. D. Walcott,1
5,British Columbia,M. V. Walcott,94
6,California,C. D. Walcott,8
7,California,M. V. Walcott,7
8,Colima,M. V. Walcott,1
9,Colorado,C. D. Walcott,2


In [9]:
#list the location and province and count of specimen for each collector 
walcott_df.groupby(['collector_name','country','province_state']).size()

collector_name  country        province_state      
C. D. Walcott   Canada         Alberta                  1
                               New Brunswick            1
                United States  Arizona                  6
                               Arkansas                 1
                               California               8
                               Colorado                 2
                               District of Columbia     1
                               Maryland                 3
                               Nevada                   3
                               New Mexico              12
                               Oklahoma                 1
                               Tennessee                7
                               Utah                     1
M. V. Walcott   Canada         Alberta                 49
                               British Columbia        94
                               Ontario                  1
                    

In [10]:
#list of countries and the province/state loaction with the frequency 
walcott_df.groupby(['country','province_state']).size()

country        province_state      
Canada         Alberta                 50
               British Columbia        94
               New Brunswick            1
               Ontario                  1
               Saskatchewan             1
Mexico         Colima                   1
Puerto Rico    Guánica                  1
United States  Arizona                 11
               Arkansas                 1
               California              15
               Colorado                 4
               District of Columbia     1
               Maryland                 3
               Nevada                   3
               New Mexico              13
               Oklahoma                 1
               South Carolina           4
               Tennessee                7
               Utah                     7
dtype: int64

In [11]:
walcott_coordinates = pd.read_csv('walcott_location.csv')
walcott_coordinates 

,country,province_state,lat,long
0,Canada,Alberta,56.529681,-112.029586
1,Canada,British Columbia,54.399416,-127.885982
2,Canada,New brunswick,47.287637,-65.885997
3,Canada,Ontario,49.927078,-85.677422
4,Canada,Saskatchewan,53.687215,-107.141410
5,Mexico,Colima,19.239392,-103.727235
6,Puerto Rico,Guanica,17.970634,-66.912939
7,United States,Arizona,34.026561,-111.877243
8,United States,Arkansas,35.453516,-92.843645
9,United States,California,37.746073,-121.822755


In [12]:
#combinding walcott_coordinates dataframe with location_walcott dataframe 
combine_walcott_location = walcott_coordinates[['country','province_state','lat','long']].merge(location_walcott_df,on= 'province_state')
combine_walcott_location


,country,province_state,lat,long,collector_name,ind_count
0,Canada,Alberta,56.529681,-112.029586,C. D. Walcott,1
1,Canada,Alberta,56.529681,-112.029586,M. V. Walcott,49
2,Canada,British Columbia,54.399416,-127.885982,M. V. Walcott,94
3,Canada,Ontario,49.927078,-85.677422,M. V. Walcott,1
4,Canada,Saskatchewan,53.687215,-107.141410,M. V. Walcott,1
5,Mexico,Colima,19.239392,-103.727235,M. V. Walcott,1
6,United States,Arizona,34.026561,-111.877243,C. D. Walcott,6
7,United States,Arizona,34.026561,-111.877243,M. V. Walcott,5
8,United States,Arkansas,35.453516,-92.843645,C. D. Walcott,1
9,United States,California,37.746073,-121.822755,C. D. Walcott,8


- ^ note: if the columns arent the same length you won't be able to<br>merge right_on= and left_on=you'll need to use on=
- resource link for merge function: https://stackoverflow.com/questions/53026827/pandas-merge-function-only-giving-column-headers-update

### Sub dataframe Mary_walcott and Charles_walcott 
- creating sub dataframe for each collector_name 
- this will give you an option to make a trace for each collector 

In [13]:
#creating mary vaux walcott sub dataframe 
mary_vaux_walcott = combine_walcott_location[combine_walcott_location['collector_name'] == 'M. V. Walcott'].copy()
mary_vaux_walcott


,country,province_state,lat,long,collector_name,ind_count
1,Canada,Alberta,56.529681,-112.029586,M. V. Walcott,49
2,Canada,British Columbia,54.399416,-127.885982,M. V. Walcott,94
3,Canada,Ontario,49.927078,-85.677422,M. V. Walcott,1
4,Canada,Saskatchewan,53.687215,-107.141410,M. V. Walcott,1
5,Mexico,Colima,19.239392,-103.727235,M. V. Walcott,1
7,United States,Arizona,34.026561,-111.877243,M. V. Walcott,5
10,United States,California,37.746073,-121.822755,M. V. Walcott,7
12,United States,Colorado,39.073178,-106.626575,M. V. Walcott,2
17,United States,New Mexico,35.929530,-106.740549,M. V. Walcott,1
19,United States,South Carolina,34.304820,-80.774066,M. V. Walcott,4


In [14]:
# creating charles doolittle walcott sub dataframe 
charles_doolittle_walcott = combine_walcott_location[combine_walcott_location['collector_name']=='C. D. Walcott'].copy()
charles_doolittle_walcott 


,country,province_state,lat,long,collector_name,ind_count
0,Canada,Alberta,56.529681,-112.029586,C. D. Walcott,1
6,United States,Arizona,34.026561,-111.877243,C. D. Walcott,6
8,United States,Arkansas,35.453516,-92.843645,C. D. Walcott,1
9,United States,California,37.746073,-121.822755,C. D. Walcott,8
11,United States,Colorado,39.073178,-106.626575,C. D. Walcott,2
13,United States,District of Columbia,38.909320,-77.014560,C. D. Walcott,1
14,United States,Maryland,39.630499,-77.387132,C. D. Walcott,3
15,United States,Nevada,39.203409,-116.386827,C. D. Walcott,3
16,United States,New Mexico,35.929530,-106.740549,C. D. Walcott,12
18,United States,Oklahoma,35.653705,-97.975609,C. D. Walcott,1


# Walcott Scatter Plot Map 
- sense we want an overall look at the different location they collected data<br>we will use go.scatter instead of mapbox because we do not need a detailed map
- we will also add the list of specimens they collected for each researcher 

In [259]:
# Walcott Scatter Plot Map 

#adding column to Mary Vaux 
mary_vaux_walcott['text']='location:'+mary_vaux_walcott['province_state']+'count: '+mary_vaux_walcott['ind_count']+'specimens: '+mary_vaux_walcott['specimen_name'].astype(str)

#adding column to Charles Doolittle 
charles_doolittle_walcott['text']='location:'+charles_doolittle_walcott['province_state']+'count: '+charles_doolittle_walcott['ind_count']+'specimens: '+charles_doolittle_walcott['specimen_name'].astype(str)

#code to create geo plot 
fig = go.Figure(data=go.Scattergeo())

#Mary Vaux Walcott 
fig.add_trace(go.Scattergeo(
            lat = mary_vaux_walcott['lat'],
            lon = mary_vaux_walcott['long'],
            text = mary_vaux_walcott['text'],
            mode = 'markers',
            marker = dict(
                size = 11,
                #color = 'steelblue', #'lightskyblue',
                line_color = 'crimson', #aqua
                line_width = 1.5,
                symbol = 35,
            ),
            name = "Mary Vaux<br>Walcott",
            opacity = 0.90
            
    ))

#Charles Doolittle Walcott 
fig.add_trace(go.Scattergeo(
            lat = charles_doolittle_walcott['lat'],
            lon = charles_doolittle_walcott['long'],
            text = charles_doolittle_walcott['text'],
            mode = 'markers',
            marker = dict(
                size = 9,
                color = 'indigo', #'lightseagreen',
                #line_color = 'purple', #'maroon', #'lightgreen',
                line_width = 1.5,
                symbol = 323,
            ),
            name = "Charles D<br>Walcott",
            opacity = 0.90
    ))

#customize layout
fig.update_layout(
        title = 'Walcott Scatter Plot Map From the Year 1900 to 1936',
        geo_scope='north america',
        #showlegend=True,
    
               
        geo = dict(
            #landcolor = "rgb(181, 130, 55)", #rich light brown
            #landcolor = "rgb(222, 206, 187)", #option2 looks grey
            #landcolor = 'wheat', #navajowhite
            
            showland = True,
            landcolor = 'burlywood',     #"rgb(212, 212, 212)",
            subunitcolor = 'cornflowerblue',      #"rgb(255, 255, 255)",
            countrycolor = 'cornflowerblue',             #"rgb(255, 255, 255)",
            showlakes = True,
            lakecolor = "antiquewhite",       #"rgb(255, 255, 255)",
            showsubunits = True,
            showcountries = True,
            showocean = False,
            #oceancolor = 'lightblue',
            resolution = 50,                #give a detailed map with state line and small island 
            projection = dict(
                type = 'conic conformal',   #type works best for north america scope
                rotation = dict(
                    lon = -100
            )
        ),
            
        lonaxis = dict(
            #showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            #showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        ),
        
            
            
            
        
           
            
        ),

    )

fig.show()
